In [1]:
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
import torch
from PIL import Image

random.seed(0)
np.random.seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import torch
from pipeline.metrics import accuracy_b, log_loss_b
from pipeline.image_extracting import extract_faces, FastMTCNN, InceptionResnetV1, process_faces, DetectionPipeline, MTCNN

transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((299, 299)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
    #torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load face detector
mtcnn = MTCNN(margin=14, keep_all=True, factor=0.5, select_largest=False, device=device).eval()

# Load facial recognition model
resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()

detection_pipeline = DetectionPipeline(detector=mtcnn, batch_size=60, n_frames=10, resize=0.25)

In [3]:
y_train = pd.read_csv('data\\metadata.csv')

In [11]:
DATA_PATH = r'D:\Machine Learning\deepfake-detection\data\validation_set'
files = os.listdir(DATA_PATH)
for i in tqdm.tqdm_notebook(range(len(files))):
    y = y_train[y_train.name == files[i].split('.')[0]].label.values[0]
    filename = DATA_PATH + '\\' + files[i]
    
    faces = detection_pipeline(filename)
    
    #print(len(faces))
    data = process_faces(faces, resnet, device, 100)
    
    with open('data\\validate.csv', 'a') as w:
        w.write(','.join([str(i) for i in data]) + ',' + str(y) + '\n')
    #break
    
# < 10 = 0, > 10 = 0

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
y0 = torch.tensor([0]*2000 + [1]*2000)
y = torch.FloatTensor([0.1]*2400 + [0.9]*1600)
log_loss_b(y, y0), 0.6931

(tensor(0.3251), 0.6931)

In [14]:
3600 / 4000

0.9

In [ ]:
real = 0
fake = 0
DATA_PATH = 'data\\other\\frames\\faces_155'
files = os.listdir(DATA_PATH)
for i in tqdm.tqdm_notebook(range(len(files))):
    y = y_train[y_train.name == files[i].split('.')[0]].label.values[0]
    if real < 2000 and y == 0:
        os.rename(DATA_PATH + '\\' + files[i], 'data\\other\\validation_set\\real\\' + files[i])
        real += 1
    if fake < 2000 and y == 1:
        os.rename(DATA_PATH + '\\' + files[i], 'data\\other\\validation_set\\fake\\' + files[i])
        fake += 1

    if fake == 2000 and real == 2000:
        print('STOP')

In [ ]:
y_train[y_train.name == 'owxbbpjpch'].label.values[0]

In [ ]:
def extract_first_frame(video_path, frames_path, y_train):
    filename = video_path.split('\\')[-1].split('.')[0]
    y = y_train[y_train.name == filename].label.values[0]
    if y == 1:
        pth = frames_path + filename + '.jpg'
        vidcap = cv2.VideoCapture(video_path)
        success,image = vidcap.read()
        cv2.imwrite(pth, cv2.resize(image, (32,32), interpolation = cv2.INTER_AREA) )  # save frame as JPEG file
    else:
        vidcap = cv2.VideoCapture(video_path)
        for i in range(5):
            pth = frames_path + filename + "." + str(i) + '.jpg'
            success,image = vidcap.read()
            cv2.imwrite(pth, cv2.resize(image, (32,32), interpolation = cv2.INTER_AREA) )  # save frame as JPEG file

In [ ]:
%%time
'''Extract first frame from each test video'''
DATA_PATH = 'data\\test_videos'
FRAMES_PATH = 'data\\test_videos_img\\img\\'
for file in os.listdir(DATA_PATH):
    if file != 'metadata.json':
        extract_first_frame(DATA_PATH + '\\' + file, FRAMES_PATH)

In [ ]:
%%time
'''Extract first frame from each train video'''
for j in tqdm.tqdm_notebook(range(17, 18)):
    DATA_PATH = 'data\\train_sets\\dfdc_train_part_' + str(j)
    FRAMES_PATH = 'data\\frames\\dfdc_train_part_'+str(j)+'_img\\'
    files = os.listdir(DATA_PATH)
    for i in tqdm.tqdm_notebook(range(len(files))):
        if files[i] != 'metadata.json':
            extract_first_frame(DATA_PATH + '\\' + files[i], FRAMES_PATH, y_train)

In [ ]:
%%time
from PIL import ImageFilter, Image
''' Apply filters to images '''
filters = [
    ImageFilter.BLUR,
    ImageFilter.CONTOUR,
    ImageFilter.DETAIL,
    ImageFilter.EDGE_ENHANCE,
    ImageFilter.EDGE_ENHANCE_MORE,
    ImageFilter.EMBOSS,
    ImageFilter.FIND_EDGES,
    ImageFilter.SMOOTH,
    ImageFilter.SMOOTH_MORE,
    ImageFilter.SHARPEN
]
for j in tqdm.tqdm_notebook(range(0, 18)):
    FRAMES_PATH = 'data\\frames\\dfdc_train_part_'+str(j)+'_img\\'
    files = os.listdir(FRAMES_PATH)
    for i in tqdm.tqdm_notebook(range(len(files))):
        if files[i] != 'metadata.json':
            for fr in filters:
                im = Image.open(FRAMES_PATH + files[i])
                im = im.filter(fr)
                im.save(FRAMES_PATH + '.'.join(files[i].split('.')[:-1]) + '.' + str(fr).split("'")[1].split('.')[-1] +  '.jpg')

In [ ]:
%%time
''' Flip images '''
from PIL import Image
for j in tqdm.tqdm_notebook(range(11, 18)):
    FRAMES_PATH = 'data\\frames\\dfdc_train_part_'+str(j)+'_img\\'
    files = os.listdir(FRAMES_PATH)
    for i in tqdm.tqdm_notebook(range(len(files))):
        if files[i] != 'metadata.json':
            im = Image.open(FRAMES_PATH + files[i])
            im = im.transpose(Image.FLIP_LEFT_RIGHT)
            im.save(FRAMES_PATH + '.'.join(files[i].split('.')[:-1]) + '.flip.jpg')

In [ ]:
%%time

'''Gather metadata'''

FRAMES_PATH = 'data\\frames\\metadata.csv'
y_data = 'name,label'
for i in tqdm.tqdm_notebook(range(0, 18)):
    DATA_PATH = 'data\\train_sets\\dfdc_train_part_' + str(i)
    for file in os.listdir(DATA_PATH):
        if file == 'metadata.json':
            with open(DATA_PATH + '\\' + file) as r:
                parsed = json.loads(r.read())
                for name in parsed:
                    y_data += '\n' + name.split('.')[0] + ',' + str(int(parsed[name]['label'] == 'FAKE'))
with open(FRAMES_PATH, 'w') as w: 
    w.write(y_data)